# E84 Robosat Guide

### This notebook is adapted from https://www.openstreetmap.org/user/daniel-j-h/diary/44321

### Prelude

Sanity check your ENV variables passed into the docker below (see README):

In [ ]:
!echo $DESIRED_ZOOM_LEVEL
!echo $PUBLIC_IP

This notebook starts with a tanzania-latest.osm.pbf file. It is an OpenStreetMap file that represents all of Tanzania. The file itself is a "Protocolbuffer Binary Format" file. More info here:

https://wiki.openstreetmap.org/wiki/PBF_Format

Osmium on our P2 Ubuntu AMI is not the latest version, so we performed the following steps locally (Macbook), then uploaded the files to S3 and pulled down on our EC2 instance.

First, install Osmium. On a Mac this is easiest with Homebrew:
```
brew install osmium-tool
```

Then, we grabbed this file from http://download.geofabrik.de/ using the command:
```
wget --limit-rate=1M http://download.geofabrik.de/africa/tanzania-latest.osm.pbf
```

Once we have the entire Tanzania OSM.PBF, we need to carve out our smaller area of interest. To do this we use the Osmium library to extract a bounding box and produce a smaller OSM.PBF that represents our AOI.

```
osmium extract --bbox '38.9410400390625,-7.0545565715284955,39.70458984374999,-5.711646879515092' tanzania-latest.osm.pbf --output map.osm.pbf
```

We copied `map.osm.pbf` from our laptop to a private S3 bucket using the AWS CLI:
```
aws s3 cp <local_path>/map.osm.pbf s3://<our_private_bucket>
```
And then onto our P2 instance (make sure IAM permissions allow your laptop to PUT and EC2 instance to GET)
```
aws s3 cp s3://<our_private_bucket>/map.osm.pbf /home/ubuntu/robosat/container_mount/
```

### Data Preparation

RoboSat comes with a tool `rs extract` to extract geometries from an OpenStreetMap base map. We will need these geometries in a minute to create labels for each building. 

This command will generate buildings.geojson with each building geometry from map.osm.pbf (our area of interest) in this format:

```
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [125.6, 10.1]
  },
  "properties": {
    "name": "Dinagat Islands"
  }
}
```

In [ ]:
!./rs extract --type building container_mount/map.osm.pbf container_mount/buildings.geojson

From that GeoJSON we need a list of all tiles with buildings in the Slippy Map filename and directory format:

`/zoom/x/y.png` (or .webp)

The resulting buildings.tiles is a CSV that looks like this (x, y, z):

```
639431,544670,19
639429,544952,19
639429,544845,19 
639429,544823,19 
...
```

More info on the SlippyMap file format here: https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames

We're using a zoom of 19 in this example but that can be changed with `$DESIRED_ZOOM_LEVEL`.

In [ ]:
!./rs cover --zoom $DESIRED_ZOOM_LEVEL container_mount/buildings.geojson container_mount/buildings.tiles

Once we have our list of SlippyMap tiles, we need to download satellite imagery for each one. `rs download` takes the buildings.tiles we created earlier and downloads a 256x256 satellite image for each one if it's available. You may see some `failed, skipping` output in the next command, that's okay as we will have plenty in our dataset. These images arrive as .webp image files in the directory specified.

**NOTE: Mapbox access token required.** (Sign-up for free at https://www.mapbox.com to get an access token)

In [ ]:
!./rs download https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.png?access_token=xxx container_mount/buildings.tiles container_mount/tiles/mapbox_satellite_tiles

For each building in buildings.tiles we also need to generate a mask. The tile size and zoom parameters must match the zoom specified earlier in the `rs cover` step (19 in our case) and the `rs download` step (a tile size of 256 is the default).

The mask is a binary representation of a feature over a background. In our case we're showing orange buildings over a denim background. This is configured in `dataset-building.toml`:

```
  # Human representation for classes.
  classes = ['background', 'building']

  # Color map for visualization and representing classes in masks.
  # Note: available colors can be found in `robosat/colors.py`
  colors  = ['denim', 'orange']
```

Satellite image and mask equivelant (from https://www.openstreetmap.org/user/daniel-j-h/diary/44321)

![satellite image](images/satellite.png)
![mask](images/mask.png)

In [ ]:
!./rs rasterize --dataset container_mount/dataset-building.toml --zoom $DESIRED_ZOOM_LEVEL --size 256 container_mount/buildings.geojson container_mount/buildings.tiles container_mount/masks

We generate our masks without regard to the aerial imagery we downloaded. It's therefor possible to have masks with no associated image because we weren't able to get an image for that tile. The number of masks and images must match exactly. Each image needs a corresponding mask to train the model.

This script removes any masks that don't have images so we're left with the same number of files in /mapbox_satellite_images and /masks. It also updates buildings.tiles by removing any tiles for which we do not have an aerial image and mask.

In [ ]:
import os
import shutil

def remove_masks(dry_run):
    csv_lines_to_remove = []
    desired_zoom_level = os.environ['DESIRED_ZOOM_LEVEL']
    buildings_tiles_path = "container_mount/buildings.tiles"
    masks_path = "container_mount/masks/" + desired_zoom_level
    satellite_images_path = "container_mount/tiles/mapbox_satellite_tiles/" + desired_zoom_level

    # Open the buildings.tiles file and get all of the lines so that we can re-write them later
    f = open(buildings_tiles_path, "r")
    lines = f.readlines()
    f.close()
    
    # for each mask directory, if we don't have an image dir remove the mask directory
    # else if we have the image directory, remove any masks that don't have an image in the image dir
    for dir in os.listdir(masks_path):
        if dir not in os.listdir(satellite_images_path):
            if dry_run:
                print("Removing mask directory: " + dir)
            else:
                shutil.rmtree('masks/' + desired_zoom_level + '/' + dir)
                
            csv_lines_to_remove.append(dir)
        else:
            # for each mask in the masks dir, check if we have an image.
            # if we don't, remove the mask
            for mask_file in os.listdir(masks_path + "/" + dir):
                file_name_only = mask_file.split(".")[0]
                image_file = file_name_only + ".webp"
                
                if image_file not in os.listdir(satellite_images_path + "/" + dir):
                    csv_line = dir + "," + file_name_only
                    csv_lines_to_remove.append(csv_line)

                    if dry_run:
                        print("Removing mask file: " + mask_file)
                    else:
                        os.remove(masks_path + "/" + dir + "/" + mask_file)
                    
    
    # We also need to remove the line from buildings.tiles if we don't have an image
    f = open(buildings_tiles_path, "w")
    for line in lines:
        skip = False
        
        for path in csv_lines_to_remove:
            if path in line:
                skip = True
        
        if not skip:
            if dry_run:
                print("Writing CSV line: " + line)
            else:
                f.write(line)
            
    f.close()


# Switch param to True to perform a dry-run and print the files that would be removed
remove_masks(False)

### Training

The following commands split our buildings.tiles set into 3 subsets:

- a training dataset on which we train the model on
- a validation dataset on which we calculate metrics on after training
- a hold-out evaluation dataset if you want to do hyper-parameter tuning

Split 80/10/10. The resulting files are:

- training.tiles
- validation.tiles
- evaluation.tiles

In [ ]:
!split -l $(expr $(cat container_mount/buildings.tiles | wc -l) \* 80 / 100) container_mount/buildings.tiles training_
!split -l $(expr $(cat training_ab | wc -l) \* 50 / 100) training_ab holdout_validation_
!mv training_aa container_mount/training.tiles
!mv holdout_validation_aa container_mount/validation.tiles
!mv holdout_validation_ab container_mount/evaluation.tiles

We then use `rs subset` to split the images and masks according to each .tiles csv we created in the previous script. The result of these commands should be a set of folders that looks like:

```
├── dataset
    └── training
        └── images
        └── labels
    └── validation
        └── images
        └── labels
    └── evaluation
        └── images
        └── labels
```

Each images and labels directory should contain .webps and .pngs for each tile in the corresponding .tiles file.



In [ ]:
!./rs subset container_mount/tiles/mapbox_satellite_tiles container_mount/validation.tiles container_mount/dataset/validation/images
!./rs subset container_mount/masks container_mount/validation.tiles container_mount/dataset/validation/labels

!./rs subset container_mount/tiles/mapbox_satellite_tiles container_mount/training.tiles container_mount/dataset/training/images
!./rs subset container_mount/masks container_mount/training.tiles container_mount/dataset/training/labels

!./rs subset container_mount/tiles/mapbox_satellite_tiles container_mount/evaluation.tiles container_mount/dataset/evaluation/images
!./rs subset container_mount/masks container_mount/evaluation.tiles container_mount/dataset/evaluation/labels

We're almost ready to start training. 

Before training the model we need to calculate the class distribution since background and building pixels are not evenly distributed in our images. `rs weights` will use the classes and dataset that we set up in dataset-buildint.toml and assign weights to each class.

In [ ]:
!./rs weights --dataset /app/container_mount/dataset-building.toml

**Important:** With the output of the `rs weights` command, update the dataset-building.toml. You will need to replace the existing values with the new ones. For example:

```
[weights]
   values = [1.615929, 5.943651]
```

Once the weights are updated in the toml file, train the model!

For the first pass you can use the parameters already set in `model-unet.toml`. One thing to double-check is the checkpoint output directory. Be sure this is pointing to a location that you can access. E.g.

`checkpoint = '/app/container_mount/checkpoints/'`

In [ ]:
!./rs train --model /app/container_mount/model-unet.toml --dataset /app/container_mount/dataset-building.toml

Training will take quite a lot of time depending on your EC2 instance. You will see the progress in the output of the previous step and, at the end of each epoch, you will see output in your checkpoint directory. In our case `/app/robosat_container_files/retraining/`.

### Prediction

Now that we have our model we can use it to visualize predictions using the serve tool. Note that you need to both export your token in MAPBOX_ACCESS_TOKEN **and as part of the mapbox url** (at the end of the `./rs serve` string)!

The checkpoint we're using here is `checkpoint-00002-of-00010.pth`. Change this to the checkpoint you would like to use for your predictions.

It is not recommended to run this if `rs train` is running unless you have an EC2 instance that can support it.

In [ ]:
!export MAPBOX_ACCESS_TOKEN=xxx && \
./rs serve --model /app/container_mount/model-unet.toml --dataset /app/container_mount/dataset-building.toml --checkpoint /app/container_mount/checkpoints/best-chkpt.pth --tile_size 256 --host 0.0.0.0 --url https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.webp?access_token=xxx

Make sure you have port 5000 open in security groups if running on EC2, and access via `http://<public_ip_or_dns>:5000`. You should see models being rendered!

### Hard-negative mining

This section walks through the steps necessary to tune the model using "negative" images. In our case that means satellite images that 100% definitely do **not** have a building in them. Our objective here is to add a set of negative images and their associated negative masks to the dataset and retrain. The result should be a better performing model with fewer false positives.

There are a couple different approaches here. From the original blog post (https://www.openstreetmap.org/user/daniel-j-h/diary/44321):

>The false positives are due to how we created the dataset: we bootstrapped a dataset based on tiles with buildings in them. Even though these tiles have some background pixels they won't contain enough background (so called negative samples) to properly learn what is not a building. If we never showed the model a single image of water it has a hard time classifying it as background.

>There are two ways for us to approach this problem:
1. add many randomly sampled background tiles to the training set, re-compute class distribution weights, then train again, or
2. use the model we trained on the bootstrapped dataset and predict on tiles where we know there are no buildings; if the model tells us there is a building put these tiles into the dataset with an all-background mask, then train again

Although you may achieve better results with option 2, we're going to demonstrate option for for simplicity.

We have created a new Robosat module called `NotBuildingHandler` which is the inverse of the building handler. Instead of extracting the geojson for buildings, it extracts the geojson for everything that is not a building. We can use this geojson file to download (what we believe will be) negative tiles. We will need to verify manually.

In [ ]:
!./rs extract --type not_building container_mount/map.osm.pbf container_mount/maybe_not_buildings.geojson

Once we have the GeoJSON, run `rs cover` to create the .tiles csv and use that to rs download all of the images from mapbox

In [ ]:
!./rs cover --zoom $DESIRED_ZOOM_LEVEL container_mount/maybe_not_buildings.geojson container_mount/maybe_not_buildings.tiles

Download all (non-building) tiles listed in our new .tiles file from Mapbox

In [ ]:
!./rs download https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.png?access_token=xxx container_mount/maybe_not_buildings.tiles container_mount/negative_mining_images

After downloading all of the possible negatives from Mapbox, manually review and select a set of negative images. For each select, add it to a new .tiles file (in our case definitely_not_buildings.tiles) and re-download only those tiles.

This is redundant. We could also copy from our already downloaded image library and save this request. This approach is taken because it presents cleaner in the notebook but feel free to do either.

In [ ]:
!./rs download https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.png?access_token=xxx container_mount/definitely_not_buildings.tiles container_mount/negative_mining_select_images

Create the masks for each negative tile in definitely_not_buildings.tiles. Note that we need to use a separate dataset-negative-buildings.toml to configure the colors correctly. If we use the original .toml configuration we'll see all orange masks because our features are not "building" but "not_building".

We have also made 100% sure that the masks generated here are going to be all negative by adding only one color to the dataset-building-negative.toml file - `colors  = ['denim', 'denim']`. As above, this could be accomplished in different ways.

In [ ]:
!./rs rasterize --dataset container_mount/dataset-building-negative.toml --zoom $DESIRED_ZOOM_LEVEL --size 256 container_mount/maybe_not_buildings.geojson container_mount/definitely_not_buildings.tiles container_mount/negative_mining_masks

Once we have our negative masks and images, we need to subset them into the existing directories. The following two blocks will create new .tiles files for only our negative samples and use those to place the images and masks in the existing /dataset/ subdirectories.

In [ ]:
!split -l $(expr $(cat container_mount/definitely_not_buildings.tiles | wc -l) \* 80 / 100) container_mount/definitely_not_buildings.tiles training_
!split -l $(expr $(cat training_ab | wc -l) \* 50 / 100) training_ab holdout_validation_
!mv training_aa negative_training.tiles
!mv holdout_validation_aa negative_validation.tiles
!mv holdout_validation_ab negative_evaluation.tiles

In [ ]:
!./rs subset container_mount/negative_mining_select_images negative_validation.tiles container_mount/dataset/validation/images
!./rs subset container_mount/negative_mining_masks negative_validation.tiles container_mount/dataset/validation/labels

!./rs subset container_mount/negative_mining_select_images negative_training.tiles container_mount/dataset/training/images
!./rs subset container_mount/negative_mining_masks negative_training.tiles container_mount/dataset/training/labels

!./rs subset container_mount/negative_mining_select_images negative_evaluation.tiles container_mount/dataset/evaluation/images
!./rs subset container_mount/negative_mining_masks negative_evaluation.tiles container_mount/dataset/evaluation/labels

Re-run rs weights with the negative tiles and masks added to the dataset. This will output an array of two numbers, add these to dataset-building.toml under [weights] as before, replacing the previous values. While you're in the .toml file, edit the `dataset = `line to point to your mounted volume so that you will not lose checkpoints once the instance is terminated.

In [ ]:
!./rs weights --dataset /app/container_mount/dataset-building.toml

Re-run train and look for an improvement!

In [ ]:
!./rs train --model /app/container_mount/model-unet.toml --dataset /app/container_mount/dataset-building.toml